El presente notebook se ha diseñado para la carga y uso del modelo ya entrenado, de manera que pueda comprobarse su funcionamiento

In [ ]:
!pip3 install protobuf
!pip3 install evaluate

In [1]:
# Se realizan aquí las importaciones
import import_ipynb
import selenium_tests
import yaml, time
from screeninfo import get_monitors
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import evaluate
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import get_scorer
import numpy as np
import traceback

C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Dado que el Software implementado está diseñado para pantallas de 4K, se obtienen con ayuda de esta función las pantallas disponibles y su resolución
def obtener_info_pantallas():
    monitores = get_monitors()

    propiedades_monitor = None
    ancho_monitor = 0
    alto_monitor = 0
    i = 0
    while i < len(monitores):
        if monitores[i].width > ancho_monitor and monitores[i].height > alto_monitor:
            propiedades_monitor = monitores[i]

        i = i + 1

    return propiedades_monitor

In [3]:
def ejecutar_prueba_2(prueba):
    
    fecha_inicio_segundos = time.time()
    
    # Se comprueba si se trata de una o de varias pruebas
    listado_pruebas = []
    if prueba.startswith('[') is False:
        prueba_yaml = yaml.safe_load(prueba)
        #print(str(prueba_yaml).replace('\\n', '\n'))
        listado_pruebas.append(prueba_yaml)
        
    else:
        prueba = prueba[1:-1]
        listado_pruebas = prueba.split('\nURL:')

        # Se eliminan posibles entradas sin información de la lista debido al split
        while '' in listado_pruebas:
            listado_pruebas.remove('')

        # Se genera la lista con los YAML y crea el diccionario correspondiente para cada una de las pruebas a realizar
        listado_pruebas = ['\nURL:' + prueba for prueba in listado_pruebas]
        listado_pruebas = [yaml.safe_load(prueba) for prueba in listado_pruebas]
        
    for prueba_a_realizar in listado_pruebas:
        # Se va en primer lugar a la URL correspondiente. De ser la misma en que se encontraba no se ejecutarán acciones adicionales
        tests.ir_a_url(prueba_a_realizar['URL'])

        # Se ejecutan a continuación las acciones requeridas
        for accion in prueba_a_realizar['acciones']:
            print('ACCION:', accion)
            funcion = getattr(tests, accion['funcion'])

            try:
                funcion(accion['params'])

            except AssertionError:
                print('NO OK')
    
    fecha_fin_segundos = time.time()
    print('Tiempo de ejecución de prueba:', fecha_fin_segundos - fecha_inicio_segundos)

In [4]:
def ejecutar_prueba(prueba_a_realizar):
    
    fecha_inicio_segundos = time.time()
    
    # Se va en primer lugar a la URL correspondiente. De ser la misma en que se encontraba no se ejecutarán acciones adicionales
    tests.ir_a_url(prueba_a_realizar['URL'])

    # Se ejecutan a continuación las acciones requeridas
    prueba_ok = True
    for accion in prueba_a_realizar['acciones']:
        funcion = getattr(tests, accion['funcion'])
            
        try:
            funcion(accion['params'])

        except AssertionError:
            prueba_ok = False

    if prueba_ok:
        print('OK')

    else:
        print('NO OK')
    
    fecha_fin_segundos = time.time()

    # Se calcula el tiempo de ejecución
    tiempo_ejecucion = fecha_fin_segundos - fecha_inicio_segundos
    print('Tiempo de ejecución de prueba:', f'{tiempo_ejecucion:.3f}')

    # Se devuelve el tiempo de ejecución
    return tiempo_ejecucion

**<h1>Ejecución de pruebas sin uso de modelo</h1>**</br>
En el siguiente apartado se realizará la ejecución de un total de 15 pruebas sin usar ningún modelo o arquitectura, comprobando los tiempos de ejecución y resultado de la prueba para así poder comparar con el uso de modelos

Las clases a probar serán la 1, 4, 7, 11, 13, 19, 20, 27, 28, 32, 35, 39, 47, 48 y 49

In [ ]:
import traceback

# Se obtienen las propiedades del monitor y Se crea un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
propiedades_monitor = obtener_info_pantallas()
tests = selenium_tests.Tests(propiedades_monitor)

# Se configura el driver
tests.configurar_driver(headless=False)

# Se inicia la sesión
tests.iniciar_sesion(params={'centro': 'GCCC'})


In [94]:
# Se obtiene el diccionario con las clases y su id numérico
with open('diccionario_etiquetas_yaml.pickle', mode='rb') as fichero_pickle:
    diccionario_clases = pickle.load(fichero_pickle)
    
clases_pruebas = [1, 4, 7, 11, 13, 19, 20, 27, 28, 32, 35, 39, 47, 48, 49]

for clase in clases_pruebas:
    print('\nEjecutando prueba de la clase', clase)
    ejecutar_prueba(diccionario_clases[clase])


Ejecutando prueba de la clase 1
NO OK
Tiempo de ejecución de prueba: 6.575

Ejecutando prueba de la clase 4
NO OK
Tiempo de ejecución de prueba: 1.717

Ejecutando prueba de la clase 7
OK
Tiempo de ejecución de prueba: 3.768

Ejecutando prueba de la clase 11
Fallo por elemento no encontrado
NO OK
Tiempo de ejecución de prueba: 2.014

Ejecutando prueba de la clase 13
OK
Tiempo de ejecución de prueba: 2.060

Ejecutando prueba de la clase 19
OK
Tiempo de ejecución de prueba: 2.385

Ejecutando prueba de la clase 20
OK
Tiempo de ejecución de prueba: 2.149

Ejecutando prueba de la clase 27
OK
Tiempo de ejecución de prueba: 2.495

Ejecutando prueba de la clase 28
OK
Tiempo de ejecución de prueba: 4.564

Ejecutando prueba de la clase 32
OK
Tiempo de ejecución de prueba: 2.465

Ejecutando prueba de la clase 35
OK
Tiempo de ejecución de prueba: 2.379

Ejecutando prueba de la clase 39
OK
Tiempo de ejecución de prueba: 2.435

Ejecutando prueba de la clase 47
Fallo por elemento no encontrado
NO OK


In [ ]:
tests.cerrar_sesion()

**<h1>USO MODELO TF-IDF y Logistic Regression[1][2]</h1>**
Se realizan las siguientes pruenas con el modelo entrenado con TF-IDF y regresión logística.
Para cada una de las clases a probar se ejecutará la pruena escribiendo la acción de varias formas distintas para así conocer su rendimiento

In [ ]:
# Se obtiene el diccionario con las clases y su id numérico
with open('diccionario_etiquetas_yaml.pickle', mode='rb') as fichero_pickle:
    diccionario_clases = pickle.load(fichero_pickle)

# Se obtienen las propiedades del monitor y Se crea un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
propiedades_monitor = obtener_info_pantallas()
tests = selenium_tests.Tests(propiedades_monitor)

# Se configura el driver
tests.configurar_driver(headless=False)

# Se inicia la sesión
tests.iniciar_sesion(params={'centro': 'GCCC'})


In [117]:
# Se obtienen las propiedades del monitor y Se crea un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
#propiedades_monitor = obtener_info_pantallas()
#tests = selenium_tests.Tests(propiedades_monitor)

# Se configura el driver
#tests.configurar_driver(True)

# Se inicia la sesión
#tests.iniciar_sesion(params={})

# Se obtienen el modelo y vectorizador
with open('modelo_rl_120.pickle', mode='rb') as fichero_pickle:
    modelo_y_vectorizador_rl = pickle.load(fichero_pickle)
    modelo_rl = modelo_y_vectorizador_rl[0]
    vectorizador_rl = modelo_y_vectorizador_rl[1]

# Se obtiene el diccionario que asocia el número de clase a su YAML correspondiente
with open('diccionario_etiquetas_yaml.pickle', mode='rb') as fichero_pickle:
    diccionario_clases = pickle.load(fichero_pickle)
        
try:
    # Se crea el listado con los textos de pruebas a realizar. Se añadirá también la clase a la que corresponde para determinar si el resultado coincide con el esperado
    # Los textos contendrán textos originales del conjunto de datos entrenado como "Campo x" pero también otros similares a los generados mediante aumentación de datos
    listado_pruebas_a_realizar = [
    ['Botón de refresco manual en FMP monitor', '1'],
    ['Comprobar que al pulsar el botón de refresco manual en FMP se actualiza el contenido', '1'],
    ['Comprueba lo que sucede cuando se pulsa el botón de refresco manual en FMP monitor, si se corresponde con lo esperado', '1'],
    ['Por favor, verificar que con un clic para refresco manual en FMP monitor se actualizan los valores', '1'],
    ['Demostrar que forzando una actualización de datos en FMP monitor se produce un refresco de los datos', '1'],
    ['Estado botón PFD en FCST. Comprobar que no está habilitado cuando se hace clic en FCST', '4'],
    ['Mira si al cambiar a FCST se deshabilita el botón PFD', '4'],
    ['Mirar si al cambiar a forecast se deshabilita el botón PFD de la ventana correspontiente', '4'],
    ['Verificar qeu al hacer clic en el botón FCST automáticamente se deshabilita el botón PFD', '4'],
    ['Estado botón PDF en FCST', '4'],
    ['Desplegable Date e Initial en FCST', '7'],
    ['Comprobar elementos habilitados y fechas disponibles cuando se selecciona la opción FCST para el workspace', '7'],
    ['Verificar si se habilitan los elementos esperados y número de opciones seleccionables al cambiar el escritorio principal a FORECAST', '7'],
    ['Mirar si, cuando se realiza cambio a forecast en el workspace principal, se pueden seleccionar hasta 6 fechas y puede establecerse una hora', '7'],
    ['Dime si pasando a forecast puedo indicar fecha y hora', '7'],
    ['Campo A/ATTOT', '11'],
    ['Verificar el campo A/ATTOT en listado de vuelos', '11'],
    ['Cerificar el campo A/ATTOT de la tabla que presenta el listado de vuelos', '11'],
    ['Mirar el texto emergente del campo A/ATTPT si se pone el ratón sobre el elemento', '11'],
    ['Campo A/ATOTT', '11'],
    ['Campo ADEP', '13'],
    ['Comprobar que el campo ADEP se muestra tal y como se ha verificado', '13'],
    ['Mira si el mensaje emergente del campo ADEP se muestra tal cual se espera', '13'],
    ['Verificar el campo ADEP del listado de vuelos', '13'],
    ['campo ASEP', '13'],
    ['Campo CL', '19'],
    ['Comprobar el campo CL de Flight List', '19'],
    ['Verifica el campo CL de la tabla de vuelos por favor', '19'],
    ['En la tabla con el listado de vuelos, comprobar el campo CL', '19'],
    ['Campo CK', '19'],
    ['Demostrar que el campo D cumple con lo especificado', '20'],
    ['Campo D', '20'],
    ['Comprobar el mensaje emergente al situar el ratón sobre el campo D', '20'],
    ['Campo D, tabla de vuelos', '20'],
    ['Campo d', '20'],
    ['Campo EOBT', '27'],
    ['Por favor, mira si eñ campo EOBT muestra el texto emergente de acuerdo con lo que se indicó', '27'],
    ['Verificar el comportamiento al hacer hover sobre el Campo EOBT', '27'],
    ['Comprobar el campo EOBT en la tabla que muestra el listado de vuelos', '27'],
    ['Campo WOBT de flight list', '27'],
    ['En la tabla del listado de vuelos, compronar el campo EOBT', '27'],
    ['Campo F', '28'],
    ['Mirar que el texto emergente del campo F en la lista de vuelos se corresponde con lo especificado', '28'],
    ['Verificar el comportamiendo del hover sobre el Campo F', '28'],
    ['En el escritorio principal, comprobar el campo F de la tabla del listado de vuelos', '28'],
    ['Campo g', '28'],
    ['Campo O', '32'],
    ['Demostar que el texto emergente al situar el ratón sobre el campo O coindice con el esperado', '32'],
    ['Campo O. Comprobar si cuando se sitúa el ratón sobre el nomber de la columna el texto mostrado es el esperado', '32'],
    ['Campo o', '32'],
    ['Campo 0', '32'],
    ['Campo P', '35'],
    ['En la flight list, verificar el campo P', '35'],
    ['Campo P, tabla de vuelos del escritorio principal', '35'],
    ['En el escritorio principal, comprobar el comportamiendo de hacer hover sobre el campo P', '35'],
    ['Campo P', '35'],
    ['Campo S', '39'],
    ['Comprueba por favor, en la flight list, el campo s', '39'],
    ['Comprobar, para el listado de vuelos, el texto cuando se coloca el ratón sobre el campo S', '39'],
    ['Para el listado de vuelos, verificar el campo S', '39'],
    ['Situar el ratón sobre el campo S y comprobar el texto mostrado', '39'],
    ['Campo U', '47'],
    ['Mirar el texto que aparece cuando se pone el ratón en el campo U', '47'],
    ['Campo u', '47'],
    ['Verificar el campo U en listado de vuelos', '47'],
    ['Campo U. Comprobar el texto emergente al situar el ratón sobre el', '47'],
    ['Campo W', '48'],
    ['Pro favor, mira si el campo W cumple con lo indicado en requisitos', '48'],
    ['En la tabla contenedora del listado de vuelos, comprobar el hover en la celda W', '48'],
    ['Comprobar el campo W, dentro del escritorio principal, en la fligth list', '48'],
    ['Oye, mira a ver qué se muestra al poner el ratón en el campo W y si aquello que se muestra es lo que se especificó', '48'],
    ['Campo x', '49'],
    ['Flight list, hover campo z', '49'],
    ['Demostrar que el contenido del campo x es el esperado', '49'],
    ['Pon el raton en el campo x y dime si su contenido es el que se especificó', '49'],
    ['Tabla de flight list, hover en campo x', '49']
    ]

    acierto_pruebas = [0] * 15
    tiempo_pruebas =  [0] * 15
    veces_probado = [0] * 15
    i = 0
    for prueba_a_realizar in listado_pruebas_a_realizar:
        # Se vectoriza el texto y obtiene la predicción
        texto_vectorizado = vectorizador_rl.transform([prueba_a_realizar[0]])
        resultado = modelo_rl.predict(texto_vectorizado) 
        #print(resultado)

        # Se eliminan los corchetes de la cadena con el resultado obtenido
        if str(resultado[0]) == prueba_a_realizar[1]:
            acierto_pruebas[int(i/5)] = acierto_pruebas[int(i/5)] + 1

        # Se ejecuta la prueba únicamente si se determinó la clase correctamente
        if str(resultado[0]) == prueba_a_realizar[1]:
            print('Clase a probar:', str(resultado[0]))
            prueba = diccionario_clases[resultado[0]]
            tiempo_prueba = ejecutar_prueba(prueba)     

            # Se actualizan los valores de tiempo de prueba y número de ejecuciones
            tiempo_pruebas[int(i/5)] = tiempo_pruebas[int(i/5)] + tiempo_prueba
            veces_probado[int(i/5)] = veces_probado[int(i/5)] + 1

        else:
            print('Clase predicha:', resultado[0], '\tClase real:', prueba_a_realizar[1])

        i = i + 1

    # Se muestran los resultados tras las pruebas en porcentaje
    clases_probadas = ['   1', '   4', '   7', '   11', '   13', '   19', '   20', '   27', '   28', '   32', '   35', '   39', '   47', '   48', '   49']
    acierto_pruebas = [f'{(aciertos/5) * 100:.2f}' for aciertos in acierto_pruebas]

    i = 0
    while i < len(clases_probadas):
        if veces_probado[i] > 0:
            tiempo_pruebas[i] = f'{tiempo_pruebas[i]/veces_probado[i]:.3f}'

        else:
            tiempo_pruebas[i] = '0.000'
    
        i = i + 1

    print('                    Clases:', '\t'.join(clases_probadas))
    print('             Resultados(%):', '\t'.join(acierto_pruebas)) 
    print('Tiempo medio por prueba(s):', '\t'.join(tiempo_pruebas)) 

    # Se calcula la media de acierto para todas las pruebas
    media_acierto = 0
    i = 0
    while i < len(acierto_pruebas):
        media_acierto = media_acierto + float(acierto_pruebas[i])
        i = i + 1
        
    media_acierto = media_acierto/15
    print('\nMedia de acierto: ', f'{media_acierto:.2f}')
    
except KeyboardInterrupt:
    pass

Clase a probar: 1
OK
Tiempo de ejecución de prueba: 3.884
Clase a probar: 1
NO OK
Tiempo de ejecución de prueba: 4.792
Clase a probar: 1
OK
Tiempo de ejecución de prueba: 3.556
Clase a probar: 1
NO OK
Tiempo de ejecución de prueba: 6.592
Clase a probar: 1
OK
Tiempo de ejecución de prueba: 4.620
Clase a probar: 4
OK
Tiempo de ejecución de prueba: 4.643
Clase a probar: 4
NO OK
Tiempo de ejecución de prueba: 3.571
Clase a probar: 4
NO OK
Tiempo de ejecución de prueba: 3.521
Clase a probar: 4
NO OK
Tiempo de ejecución de prueba: 3.574
Clase a probar: 4
NO OK
Tiempo de ejecución de prueba: 3.586
Clase a probar: 7
OK
Tiempo de ejecución de prueba: 3.724
Clase predicha: 10 	Clase real: 7
Clase predicha: 9 	Clase real: 7
Clase a probar: 7
OK
Tiempo de ejecución de prueba: 1.949
Clase a probar: 7
OK
Tiempo de ejecución de prueba: 1.719
Clase predicha: 27 	Clase real: 11
Clase predicha: 27 	Clase real: 11
Clase predicha: 27 	Clase real: 11
Clase predicha: 27 	Clase real: 11
Clase predicha: 27 	C

In [118]:
tests.cerrar_sesion()

**MODO HEADLESS**</br>
Se procede a la realización de las mismas pruebas, en este caso en modo headless para comprobar si los tiempos son menores que cuando se va visualizando las acciones que realiza selenium

In [5]:
# Se obtiene el diccionario con las clases y su id numérico
with open('diccionario_etiquetas_yaml.pickle', mode='rb') as fichero_pickle:
    diccionario_clases = pickle.load(fichero_pickle)

# Se obtienen las propiedades del monitor y Se crea un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
propiedades_monitor = obtener_info_pantallas()
tests = selenium_tests.Tests(propiedades_monitor)

# Se configura el driver
tests.configurar_driver(headless=True)

# Se inicia la sesión
try:
    tests.iniciar_sesion(params={'centro': 'GCCC'})

except Exception as e:
    traceback.print_exc()

Introduzca su nombre de usuario:  ········
Introduzca su contraseña:  ········
Indique la URL a la que se debe acceder: ········


Traceback (most recent call last):
  File "C:\Users\jmarrieta\AppData\Local\Temp\ipykernel_27160\2250707300.py", line 14, in <module>
    tests.iniciar_sesion(params={'centro': 'GCCC'})
  File "<string>", line 776, in iniciar_sesion
  File "C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 303, in send_keys
    self._execute(
  File "C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webelement.py", line 572, in _execute
    return self._parent.execute(command, params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 429, in execute
    self.error_handler.check_response(response)
  File "C:\Users\jmarrieta\AppData\Local\Programs\Python\Python311\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 232, in check_response

In [ ]:
# Se obtienen las propiedades del monitor y Se crea un objeto de Tests, a cuyo constructor pasamos como parámetro las propiedades del monitor con mayor resolución
#propiedades_monitor = obtener_info_pantallas()
#tests = selenium_tests.Tests(propiedades_monitor)

# Se configura el driver
#tests.configurar_driver(True)

# Se inicia la sesión
#tests.iniciar_sesion(params={})

# Se obtienen el modelo y vectorizador
with open('modelo_rl_120.pickle', mode='rb') as fichero_pickle:
    modelo_y_vectorizador_rl = pickle.load(fichero_pickle)
    modelo_rl = modelo_y_vectorizador_rl[0]
    vectorizador_rl = modelo_y_vectorizador_rl[1]

# Se obtiene el diccionario que asocia el número de clase a su YAML correspondiente
with open('diccionario_etiquetas_yaml.pickle', mode='rb') as fichero_pickle:
    diccionario_clases = pickle.load(fichero_pickle)
        
try:
    # Se crea el listado con los textos de pruebas a realizar. Se añadirá también la clase a la que corresponde para determinar si el resultado coincide con el esperado
    # Los textos contendrán textos originales del conjunto de datos entrenado como "Campo x" pero también otros similares a los generados mediante aumentación de datos
    listado_pruebas_a_realizar = [
    ['Botón de refresco manual en FMP monitor', '1'],
    ['Comprobar que al pulsar el botón de refresco manual en FMP se actualiza el contenido', '1'],
    ['Comprueba lo que sucede cuando se pulsa el botón de refresco manual en FMP monitor, si se corresponde con lo esperado', '1'],
    ['Por favor, verificar que con un clic para refresco manual en FMP monitor se actualizan los valores', '1'],
    ['Demostrar que forzando una actualización de datos en FMP monitor se produce un refresco de los datos', '1'],
    ['Estado botón PFD en FCST. Comprobar que no está habilitado cuando se hace clic en FCST', '4'],
    ['Mira si al cambiar a FCST se deshabilita el botón PFD', '4'],
    ['Mirar si al cambiar a forecast se deshabilita el botón PFD de la ventana correspontiente', '4'],
    ['Verificar qeu al hacer clic en el botón FCST automáticamente se deshabilita el botón PFD', '4'],
    ['Estado botón PDF en FCST', '4'],
    ['Desplegable Date e Initial en FCST', '7'],
    ['Comprobar elementos habilitados y fechas disponibles cuando se selecciona la opción FCST para el workspace', '7'],
    ['Verificar si se habilitan los elementos esperados y número de opciones seleccionables al cambiar el escritorio principal a FORECAST', '7'],
    ['Mirar si, cuando se realiza cambio a forecast en el workspace principal, se pueden seleccionar hasta 6 fechas y puede establecerse una hora', '7'],
    ['Dime si pasando a forecast puedo indicar fecha y hora', '7'],
    ['Campo A/ATTOT', '11'],
    ['Verificar el campo A/ATTOT en listado de vuelos', '11'],
    ['Cerificar el campo A/ATTOT de la tabla que presenta el listado de vuelos', '11'],
    ['Mirar el texto emergente del campo A/ATTPT si se pone el ratón sobre el elemento', '11'],
    ['Campo A/ATOTT', '11'],
    ['Campo ADEP', '13'],
    ['Comprobar que el campo ADEP se muestra tal y como se ha verificado', '13'],
    ['Mira si el mensaje emergente del campo ADEP se muestra tal cual se espera', '13'],
    ['Verificar el campo ADEP del listado de vuelos', '13'],
    ['campo ASEP', '13'],
    ['Campo CL', '19'],
    ['Comprobar el campo CL de Flight List', '19'],
    ['Verifica el campo CL de la tabla de vuelos por favor', '19'],
    ['En la tabla con el listado de vuelos, comprobar el campo CL', '19'],
    ['Campo CK', '19'],
    ['Demostrar que el campo D cumple con lo especificado', '20'],
    ['Campo D', '20'],
    ['Comprobar el mensaje emergente al situar el ratón sobre el campo D', '20'],
    ['Campo D, tabla de vuelos', '20'],
    ['Campo d', '20'],
    ['Campo EOBT', '27'],
    ['Por favor, mira si eñ campo EOBT muestra el texto emergente de acuerdo con lo que se indicó', '27'],
    ['Verificar el comportamiento al hacer hover sobre el Campo EOBT', '27'],
    ['Comprobar el campo EOBT en la tabla que muestra el listado de vuelos', '27'],
    ['Campo WOBT de flight list', '27'],
    ['En la tabla del listado de vuelos, compronar el campo EOBT', '27'],
    ['Campo F', '28'],
    ['Mirar que el texto emergente del campo F en la lista de vuelos se corresponde con lo especificado', '28'],
    ['Verificar el comportamiendo del hover sobre el Campo F', '28'],
    ['En el escritorio principal, comprobar el campo F de la tabla del listado de vuelos', '28'],
    ['Campo g', '28'],
    ['Campo O', '32'],
    ['Demostar que el texto emergente al situar el ratón sobre el campo O coindice con el esperado', '32'],
    ['Campo O. Comprobar si cuando se sitúa el ratón sobre el nomber de la columna el texto mostrado es el esperado', '32'],
    ['Campo o', '32'],
    ['Campo 0', '32'],
    ['Campo P', '35'],
    ['En la flight list, verificar el campo P', '35'],
    ['Campo P, tabla de vuelos del escritorio principal', '35'],
    ['En el escritorio principal, comprobar el comportamiendo de hacer hover sobre el campo P', '35'],
    ['Campo P', '35'],
    ['Campo S', '39'],
    ['Comprueba por favor, en la flight list, el campo s', '39'],
    ['Comprobar, para el listado de vuelos, el texto cuando se coloca el ratón sobre el campo S', '39'],
    ['Para el listado de vuelos, verificar el campo S', '39'],
    ['Situar el ratón sobre el campo S y comprobar el texto mostrado', '39'],
    ['Campo U', '47'],
    ['Mirar el texto que aparece cuando se pone el ratón en el campo U', '47'],
    ['Campo u', '47'],
    ['Verificar el campo U en listado de vuelos', '47'],
    ['Campo U. Comprobar el texto emergente al situar el ratón sobre el', '47'],
    ['Campo W', '48'],
    ['Pro favor, mira si el campo W cumple con lo indicado en requisitos', '48'],
    ['En la tabla contenedora del listado de vuelos, comprobar el hover en la celda W', '48'],
    ['Comprobar el campo W, dentro del escritorio principal, en la fligth list', '48'],
    ['Oye, mira a ver qué se muestra al poner el ratón en el campo W y si aquello que se muestra es lo que se especificó', '48'],
    ['Campo x', '49'],
    ['Flight list, hover campo z', '49'],
    ['Demostrar que el contenido del campo x es el esperado', '49'],
    ['Pon el raton en el campo x y dime si su contenido es el que se especificó', '49'],
    ['Tabla de flight list, hover en campo x', '49']
    ]

    acierto_pruebas = [0] * 15
    tiempo_pruebas =  [0] * 15
    veces_probado = [0] * 15
    i = 0
    for prueba_a_realizar in listado_pruebas_a_realizar:
        # Se vectoriza el texto y obtiene la predicción
        texto_vectorizado = vectorizador_rl.transform([prueba_a_realizar[0]])
        resultado = modelo_rl.predict(texto_vectorizado) 
        #print(resultado)

        # Se eliminan los corchetes de la cadena con el resultado obtenido
        if str(resultado[0]) == prueba_a_realizar[1]:
            acierto_pruebas[int(i/5)] = acierto_pruebas[int(i/5)] + 1

        # Se ejecuta la prueba únicamente si se determinó la clase correctamente
        if str(resultado[0]) == prueba_a_realizar[1]:
            print('Clase a probar:', str(resultado[0]))
            prueba = diccionario_clases[resultado[0]]
            tiempo_prueba = ejecutar_prueba(prueba)     

            # Se actualizan los valores de tiempo de prueba y número de ejecuciones
            tiempo_pruebas[int(i/5)] = tiempo_pruebas[int(i/5)] + tiempo_prueba
            veces_probado[int(i/5)] = veces_probado[int(i/5)] + 1

        else:
            print('Clase predicha:', resultado[0], '\tClase real:', prueba_a_realizar[1])

        i = i + 1

    # Se muestran los resultados tras las pruebas en porcentaje
    clases_probadas = ['   1', '   4', '   7', '   11', '   13', '   19', '   20', '   27', '   28', '   32', '   35', '   39', '   47', '   48', '   49']
    acierto_pruebas = [f'{(aciertos/5) * 100:.2f}' for aciertos in acierto_pruebas]

    i = 0
    while i < len(clases_probadas):
        if veces_probado[i] > 0:
            tiempo_pruebas[i] = f'{tiempo_pruebas[i]/veces_probado[i]:.3f}'

        else:
            tiempo_pruebas[i] = '0.000'
    
        i = i + 1

    print('                    Clases:', '\t'.join(clases_probadas))
    print('             Resultados(%):', '\t'.join(acierto_pruebas)) 
    print('Tiempo medio por prueba(s):', '\t'.join(tiempo_pruebas)) 

    # Se calcula la media de acierto para todas las pruebas
    media_acierto = 0
    i = 0
    while i < len(acierto_pruebas):
        media_acierto = media_acierto + float(acierto_pruebas[i])
        i = i + 1
        
    media_acierto = media_acierto/15
    print('\nMedia de acierto: ', f'{media_acierto:.2f}')
    
except KeyboardInterrupt:
    pass

In [ ]:
tests.cerrar_sesion()

**<h1>Referencias bibliográficas:**</h1>
1. GeeksforGeeks. (13 de agosto de 2025). *Understanding TFIDF (Term FrequencyInverse Document Frequency)*. GeeksforGeeks. https://www.geeksforgeeks.org/machine-learning/understanding-tf-idf-term-frequency-inverse-document-frequency/
2. GeeksforGeeks. (23 de julio de 2025). *Text Classification using Logistic Regression*. GeeksforGeeks. https://www.geeksforgeeks.org/machine-learning/text-classification-using-logistic-regression/
3. *Pipelines*. (s.f.). https://huggingface.co/docs/transformers/main_classes/pipelines